# 🏛️ Pyramid Perspective Correction - Teotihuacan

This notebook transforms aerial pyramid images to create a frontal view perspective using OpenCV perspective correction.

**Google Colab Compatible** 📱 (No interactive clicking required)

---

In [ ]:
# Install required packages (if not already installed)
!pip install opencv-python matplotlib numpy pillow

# Import necessary libraries
import cv2
import numpy as np
import matplotlib.pyplot as plt
from google.colab import files
import io
from PIL import Image
import os

print("✅ All packages installed and imported successfully!")

In [ ]:
def upload_image():
    """
    Upload an image file in Google Colab
    """
    print("📸 Please upload your Teotihuacan pyramid image:")
    uploaded = files.upload()
    
    # Get the uploaded filename
    filename = list(uploaded.keys())[0]
    print(f"✅ Uploaded: {filename}")
    
    return filename

# Upload your image
image_filename = upload_image()

In [ ]:
# Load and display the uploaded image with coordinate grid
img = cv2.imread(image_filename)
img_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
height, width = img_rgb.shape[:2]

def show_image_with_grid(image, grid_step=50):
    """Display image with coordinate grid overlay"""
    fig, ax = plt.subplots(figsize=(15, 10))
    ax.imshow(image)
    
    # Add grid lines
    for x in range(0, width, grid_step):
        ax.axvline(x=x, color='white', alpha=0.3, linewidth=0.5)
    for y in range(0, height, grid_step):
        ax.axhline(y=y, color='white', alpha=0.3, linewidth=0.5)
    
    # Add coordinate labels
    for x in range(0, width, grid_step*2):
        ax.text(x, 20, str(x), color='yellow', fontsize=8, weight='bold', 
               bbox=dict(boxstyle="round,pad=0.2", facecolor="black", alpha=0.7))
    for y in range(0, height, grid_step*2):
        ax.text(10, y, str(y), color='yellow', fontsize=8, weight='bold',
               bbox=dict(boxstyle="round,pad=0.2", facecolor="black", alpha=0.7))
    
    ax.set_title("📷 Original Image with Coordinate Grid\n" + 
                "Use this grid to identify the coordinates of pyramid corners", 
                fontsize=16, pad=20)
    ax.set_xlabel(f"X coordinates (0 to {width})", fontsize=12)
    ax.set_ylabel(f"Y coordinates (0 to {height})", fontsize=12)
    
    plt.tight_layout()
    plt.show()

show_image_with_grid(img_rgb)

print(f"📐 Image dimensions: {width} x {height} pixels")
print("💡 Use the coordinate grid to identify the pyramid corner coordinates")
print("📍 You'll need 4 points: top-left, top-right, bottom-right, bottom-left of the pyramid base")

In [ ]:
# Manual coordinate input (reliable method for Colab)
print("🎯 Enter the coordinates of the 4 pyramid base corners:")
print("📍 Look at the grid above to identify the coordinates")
print("⚠️  Order is important: top-left → top-right → bottom-right → bottom-left")
print()

# You can modify these coordinates based on your image
# Default coordinates (adjust these based on your pyramid location)
default_coords = [
    [width * 0.35, height * 0.45],  # Top-left
    [width * 0.65, height * 0.45],  # Top-right
    [width * 0.75, height * 0.85],  # Bottom-right
    [width * 0.25, height * 0.85]   # Bottom-left
]

print("📝 Default coordinates (you can modify these):")
for i, coord in enumerate(default_coords):
    labels = ['top-left', 'top-right', 'bottom-right', 'bottom-left']
    print(f"  {i+1}. {labels[i]}: [{coord[0]:.0f}, {coord[1]:.0f}]")

print("\n" + "="*50)
print("✏️  MODIFY THE COORDINATES BELOW:")
print("="*50)

# Manual coordinate input - MODIFY THESE VALUES
# Look at your image and the grid, then update these coordinates
selected_points = np.float32([
    [width * 0.35, height * 0.45],  # 1. Top-left corner of pyramid base
    [width * 0.65, height * 0.45],  # 2. Top-right corner of pyramid base
    [width * 0.75, height * 0.85],  # 3. Bottom-right corner of pyramid base
    [width * 0.25, height * 0.85]   # 4. Bottom-left corner of pyramid base
])

# Alternative: Use specific pixel coordinates if you prefer
# Uncomment and modify the lines below if you want to use exact pixel coordinates:

# selected_points = np.float32([
#     [350, 200],  # 1. Top-left corner (x, y)
#     [650, 200],  # 2. Top-right corner (x, y)
#     [750, 400],  # 3. Bottom-right corner (x, y)
#     [250, 400]   # 4. Bottom-left corner (x, y)
# ])

print("\n✅ Coordinates set:")
labels = ['top-left', 'top-right', 'bottom-right', 'bottom-left']
for i, point in enumerate(selected_points):
    print(f"  {i+1}. {labels[i]}: [{point[0]:.0f}, {point[1]:.0f}]")

In [ ]:
# Preview the selected points on the image
def preview_selected_points(image, points):
    """Show the image with selected points marked"""
    fig, ax = plt.subplots(figsize=(14, 10))
    ax.imshow(image)
    
    # Plot the selected points
    ax.scatter(points[:, 0], points[:, 1], c='red', s=150, zorder=5, 
               edgecolors='white', linewidth=3, alpha=0.9)
    
    # Draw lines connecting the points
    x_coords = [p[0] for p in points] + [points[0][0]]
    y_coords = [p[1] for p in points] + [points[0][1]]
    ax.plot(x_coords, y_coords, 'r-', linewidth=3, alpha=0.8)
    
    # Add point labels
    labels = ['1\n(top-left)', '2\n(top-right)', '3\n(bottom-right)', '4\n(bottom-left)']
    for i, point in enumerate(points):
        ax.text(point[0] + 20, point[1] - 30, labels[i], fontsize=14, color='red', 
                weight='bold', bbox=dict(boxstyle="round,pad=0.4", facecolor="white", alpha=0.9))
    
    ax.set_title('🎯 Preview: Selected Points for Perspective Correction', fontsize=16, pad=20)
    ax.axis('off')
    plt.tight_layout()
    plt.show()

preview_selected_points(img_rgb, selected_points)
print("👀 Review the selected points above")
print("⚠️  If they don't look right, go back to the previous cell and modify the coordinates")

In [ ]:
def correct_pyramid_perspective(image_path, src_points, output_filename="corrected_pyramid.jpg"):
    """
    Transform the perspective of a pyramid image to make it appear as if 
    the focal plane is parallel to the pyramid's front face.
    """
    
    # Load the image
    img = cv2.imread(image_path)
    if img is None:
        raise ValueError("Could not load image. Check the file path.")
    
    # Convert BGR to RGB for matplotlib display
    img_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    
    # Get image dimensions
    height, width = img.shape[:2]
    
    # Define destination points (rectangular perspective)
    margin = 100
    dst_points = np.float32([
        [margin, margin],                    # Top-left
        [width - margin, margin],            # Top-right
        [width - margin, height - margin],   # Bottom-right
        [margin, height - margin]            # Bottom-left
    ])
    
    # Calculate the perspective transformation matrix
    matrix = cv2.getPerspectiveTransform(src_points, dst_points)
    
    # Apply the perspective transformation
    corrected_img = cv2.warpPerspective(img_rgb, matrix, (width, height))
    
    # Save the corrected image
    corrected_bgr = cv2.cvtColor(corrected_img, cv2.COLOR_RGB2BGR)
    cv2.imwrite(output_filename, corrected_bgr)
    
    return corrected_img, matrix, src_points, dst_points

# Apply perspective correction
try:
    corrected_img, transformation_matrix, src_pts, dst_pts = correct_pyramid_perspective(
        image_filename, 
        selected_points, 
        "corrected_pyramid.jpg"
    )
    print("✅ Perspective correction applied successfully!")
except Exception as e:
    print(f"❌ Error: {e}")

In [ ]:
# Display the results side by side
fig, axes = plt.subplots(1, 2, figsize=(20, 10))

# Original image with source points marked
axes[0].imshow(img_rgb)
axes[0].plot([src_pts[0][0], src_pts[1][0], src_pts[2][0], src_pts[3][0], src_pts[0][0]], 
             [src_pts[0][1], src_pts[1][1], src_pts[2][1], src_pts[3][1], src_pts[0][1]], 
             'r-', linewidth=3, label='Source area', alpha=0.8)
axes[0].scatter(src_pts[:, 0], src_pts[:, 1], c='red', s=120, zorder=5, edgecolors='white', linewidth=2)

# Add point labels
labels = ['1', '2', '3', '4']
for i, point in enumerate(src_pts):
    axes[0].text(point[0] + 20, point[1] - 20, labels[i], fontsize=16, color='red', 
                weight='bold', bbox=dict(boxstyle="circle,pad=0.3", facecolor="white", alpha=0.9))

axes[0].set_title('📷 Original Image with Source Points', fontsize=16, pad=20)
axes[0].legend(fontsize=12)
axes[0].axis('off')

# Corrected image with destination points marked
axes[1].imshow(corrected_img)
axes[1].plot([dst_pts[0][0], dst_pts[1][0], dst_pts[2][0], dst_pts[3][0], dst_pts[0][0]], 
             [dst_pts[0][1], dst_pts[1][1], dst_pts[2][1], dst_pts[3][1], dst_pts[0][1]], 
             'g-', linewidth=3, label='Destination area', alpha=0.8)
axes[1].scatter(dst_pts[:, 0], dst_pts[:, 1], c='green', s=120, zorder=5, edgecolors='white', linewidth=2)

axes[1].set_title('🎯 Perspective Corrected Image', fontsize=16, pad=20)
axes[1].legend(fontsize=12)
axes[1].axis('off')

plt.tight_layout()
plt.show()

print("🎉 Transformation complete!")
print(f"📁 Corrected image saved as: corrected_pyramid.jpg")

In [ ]:
# Download the corrected image
def download_corrected_image():
    """
    Download the corrected image to your local machine
    """
    try:
        files.download('corrected_pyramid.jpg')
        print("✅ Download started! Check your Downloads folder.")
    except Exception as e:
        print(f"❌ Download error: {e}")

# Download the corrected image
download_corrected_image()

print("💾 Your perspective-corrected pyramid image has been downloaded!")

In [ ]:
# Coordinate helper - shows different coordinate options
def show_coordinate_examples():
    """Show example coordinates for different scenarios"""
    print("📍 COORDINATE HELPER - Example coordinates for common scenarios:")
    print("="*60)
    
    # Example 1: Pyramid in center
    print("\n🎯 Example 1: Pyramid centered in image")
    center_coords = np.float32([
        [width * 0.3, height * 0.4],   # Top-left
        [width * 0.7, height * 0.4],   # Top-right
        [width * 0.8, height * 0.8],   # Bottom-right
        [width * 0.2, height * 0.8]    # Bottom-left
    ])
    for i, coord in enumerate(center_coords):
        labels = ['top-left', 'top-right', 'bottom-right', 'bottom-left']
        print(f"  [{coord[0]:.0f}, {coord[1]:.0f}],  # {labels[i]}")
    
    # Example 2: Pyramid on left side
    print("\n🎯 Example 2: Pyramid on left side of image")
    left_coords = np.float32([
        [width * 0.1, height * 0.3],   # Top-left
        [width * 0.5, height * 0.3],   # Top-right
        [width * 0.6, height * 0.7],   # Bottom-right
        [width * 0.05, height * 0.7]   # Bottom-left
    ])
    for i, coord in enumerate(left_coords):
        labels = ['top-left', 'top-right', 'bottom-right', 'bottom-left']
        print(f"  [{coord[0]:.0f}, {coord[1]:.0f}],  # {labels[i]}")
    
    # Example 3: Large pyramid
    print("\n🎯 Example 3: Large pyramid filling most of image")
    large_coords = np.float32([
        [width * 0.2, height * 0.2],   # Top-left
        [width * 0.8, height * 0.2],   # Top-right
        [width * 0.9, height * 0.9],   # Bottom-right
        [width * 0.1, height * 0.9]    # Bottom-left
    ])
    for i, coord in enumerate(large_coords):
        labels = ['top-left', 'top-right', 'bottom-right', 'bottom-left']
        print(f"  [{coord[0]:.0f}, {coord[1]:.0f}],  # {labels[i]}")
    
    print("\n💡 Copy and paste the coordinates that best match your pyramid into Cell 4!")

# Show coordinate examples
show_coordinate_examples()

In [ ]:
# Fine-tuning with different destination rectangles
def fine_tune_perspective_correction(image_path, src_points, scale_factor=0.8, margin_factor=0.1):
    """
    Fine-tune the perspective correction with different destination options
    """
    img = cv2.imread(image_path)
    img_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    height, width = img.shape[:2]
    
    # Calculate destination rectangle
    margin = int(min(width, height) * margin_factor)
    center_x, center_y = width // 2, height // 2
    dst_width = int(width * scale_factor)
    dst_height = int(height * scale_factor)
    
    dst_points = np.float32([
        [center_x - dst_width//2, center_y - dst_height//2],   # Top-left
        [center_x + dst_width//2, center_y - dst_height//2],   # Top-right
        [center_x + dst_width//2, center_y + dst_height//2],   # Bottom-right
        [center_x - dst_width//2, center_y + dst_height//2]    # Bottom-left
    ])
    
    matrix = cv2.getPerspectiveTransform(src_points, dst_points)
    corrected_img = cv2.warpPerspective(img_rgb, matrix, (width, height))
    
    return corrected_img, matrix

# Fine-tune with different parameters
print("🔧 Fine-tuning perspective correction with different parameters:")

# Different scale factors
scale_factors = [0.6, 0.8, 1.0]

fig, axes = plt.subplots(1, len(scale_factors), figsize=(18, 6))

for i, scale in enumerate(scale_factors):
    fine_tuned_img, _ = fine_tune_perspective_correction(image_filename, selected_points, scale)
    axes[i].imshow(fine_tuned_img)
    axes[i].set_title(f'Scale: {scale}', fontsize=14)
    axes[i].axis('off')

plt.suptitle('🔧 Fine-Tuned Perspective Corrections', fontsize=16, y=1.02)
plt.tight_layout()
plt.show()

print("🎛️ Different scale factors:")
print("   • 0.6 = More zoomed in (less surrounding area)")
print("   • 0.8 = Balanced view")
print("   • 1.0 = Full image utilization")
print("\n💡 Choose the scale that gives you the best result!")

## 📋 Simplified Usage Instructions

### 🚀 **Step-by-Step Guide:**

1. **📸 Upload Image** (Cell 2)
   - Upload your Teotihuacan pyramid image

2. **📐 View Grid** (Cell 3)
   - Look at your image with coordinate grid overlay
   - Note the coordinates of pyramid corners

3. **✏️ Set Coordinates** (Cell 4)
   - **This is the key step!**
   - Modify the coordinates in the `selected_points` array
   - Use the grid from step 2 to identify the right coordinates

4. **👀 Preview** (Cell 5)
   - Check if your selected points look correct
   - If not, go back to Cell 4 and adjust

5. **⚙️ Apply Correction** (Cell 6)
   - Runs the perspective transformation

6. **📊 View Results** (Cell 7)
   - See before/after comparison

7. **💾 Download** (Cell 8)
   - Get your corrected image

---

### 💡 **Pro Tips:**

- **Use Cell 9** for coordinate examples if you're unsure
- **Grid coordinates**: The white grid helps you identify pixel positions
- **Order matters**: Always go top-left → top-right → bottom-right → bottom-left
- **Preview first**: Always check Cell 5 before applying correction
- **Experiment**: Try Cell 10 for different scale factors

---

### 🎯 **No More Crashes!**

This version uses manual coordinate input instead of interactive clicking, so it works reliably in Google Colab without any crashes.

**Happy pyramid correcting! 🏛️✨**